In [2]:
import pandas as pd
import json

In [4]:
with open("../../ti/data_new/western_africa_ys_nodups.json", "r") as f:
    labs = json.load(f)
len(labs)

12671

In [2]:
df = pd.read_csv("./models/phl_v2/kfold0/results/epoch101_preds.csv")
df.head()

,pred,label,name
0,155.722855,105.30,/sciclone/geograd/heather_data/imprecision/sch...
1,123.611137,132.51,/sciclone/geograd/heather_data/imprecision/sch...
2,146.468307,172.25,/sciclone/geograd/heather_data/imprecision/sch...
3,149.179749,133.93,/sciclone/geograd/heather_data/imprecision/sch...
4,160.936981,93.14,/sciclone/geograd/heather_data/imprecision/sch...


In [3]:
df.shape

(1686, 3)

In [3]:
df["name"][0]

'/sciclone/geograd/heather_data/imprecision/schools/imagery/clipped/PH/107022.tiff'

In [4]:
df2 = pd.read_csv("./models/phl_v2/kfold0/phl_results/epoch101_target_dup_preds_new.csv")
df2.head()

,pred,label,name
0,162.402084,105.3,/sciclone/geograd/heather_data/imprecision/sch...
1,152.662704,105.3,/sciclone/geograd/heather_data/imprecision/sch...
2,154.639465,105.3,/sciclone/geograd/heather_data/imprecision/sch...
3,158.953308,105.3,/sciclone/geograd/heather_data/imprecision/sch...
4,155.487823,105.3,/sciclone/geograd/heather_data/imprecision/sch...


In [5]:
df2["name"][0]

'/sciclone/geograd/heather_data/imprecision/schools/imagery/dup_clips/PH/107022_1.tiff'

In [16]:
import pandas as pd
import numpy as np
import json
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from copy import deepcopy

In [17]:
nodups_preds = "./models/phl_v2/kfold0/results/epoch101_preds.csv"
dups_preds = "./models/phl_v2/kfold0/phl_results/epoch101_target_dup_preds_new.csv"
fcs_path = "./models/phl_v2/kfold0/phl_results/dup_tsne_result_1d_p10.json"

In [18]:
# Function to calculate r2 and RMSE
def r2_rmse_og(g):
    r2 = r2_score(g['mean_true_y'], g['avg_pred_y'])
    rmse = np.sqrt(mean_absolute_error(g['mean_true_y'], g['avg_pred_y']))
    return pd.Series(dict(r2=r2, rmse=rmse))

# Load labels, coordinates, and train labels
with open("/sciclone/geograd/heather_data/imprecision/schools/data/phl_dup_ys.json", "r") as f:
    labels = json.load(f)

with open("/sciclone/geograd/heather_data/imprecision/schools/data/phl_dup_coords.json", "r") as f:
    coords = json.load(f)

with open(fcs_path, "r") as f:
    fcs = json.load(f)

# Load labels, coordinates, and train labels
with open("/sciclone/geograd/heather_data/imprecision/schools/data/phl_ys.json", "r") as f:
    labels2 = json.load(f)

with open("/sciclone/geograd/heather_data/imprecision/schools/data/phl_coords.json", "r") as f:
    coords2 = json.load(f)

coords.update(coords2)
labels.update(labels2)

In [19]:
# Load validation IDs
val_ids = pd.read_csv(nodups_preds)
val_ids["DHSID"] = val_ids["name"].str.split("/").str[-1].str.split("_").str[0].str.split(".").str[0]
val_ids = val_ids["DHSID"].unique().tolist()

In [20]:
val_ids[0:5]

['107022', '107027', '107028', '107029', '107032']

In [24]:
# Load and prepare the predictions data
def prepare_df(filepath):
    df = pd.read_csv(filepath)
    print(df.shape)
    df["label"] = df["name"].map(labels)
    df["DHSID"] = df["name"].str.split("/").str[-1].str.split("_").str[0].str.split(".").str[0]
    df["coords"] = df["name"].map(coords)
    df["lon"] = df["coords"].str[0]
    df["lat"] = df["coords"].str[1]

    df["fc"] = df["name"].map(fcs)
    df["fc"] = df["fc"].str[0].astype(float)
    
    return df

# df = prepare_df(dups_preds)
# df.head()

In [25]:
df = prepare_df(dups_preds)
df2 = prepare_df(nodups_preds)
df = pd.concat([df, df2], ignore_index=True)
print(df.shape)
df = df.dropna()
print(df.shape)

(50473, 3)
(1686, 3)
(52159, 8)
(52159, 8)


In [12]:
df = df.dropna()
print(df.shape)

# Split into training and validation sets based on val_ids
train_df = df[~df["DHSID"].isin(val_ids)].copy()
test_df = df[df["DHSID"].isin(val_ids)].copy()

(50473, 8)


In [13]:
train_df

,pred,label,name,DHSID,coords,lon,lat,fc
10,150.792999,137.05,/sciclone/geograd/heather_data/imprecision/sch...,107023,"[120.60216962040364, 15.129535382173188]",120.602170,15.129535,49.723133
11,142.886948,137.05,/sciclone/geograd/heather_data/imprecision/sch...,107023,"[120.59703367387488, 15.13247497416016]",120.597034,15.132475,58.453331
12,141.133469,137.05,/sciclone/geograd/heather_data/imprecision/sch...,107023,"[120.59570872142545, 15.125129975727976]",120.595709,15.125130,57.799820
13,138.692993,137.05,/sciclone/geograd/heather_data/imprecision/sch...,107023,"[120.59667314769688, 15.134054079392921]",120.596673,15.134054,59.324184
14,124.932808,137.05,/sciclone/geograd/heather_data/imprecision/sch...,107023,"[120.59120220116876, 15.128318350552481]",120.591202,15.128318,59.368820
...,...,...,...,...,...,...,...,...
50458,119.964935,123.13,/sciclone/geograd/heather_data/imprecision/sch...,136826,"[120.99277972066396, 14.678938188989163]",120.992780,14.678938,-3.959799
50459,119.940262,123.13,/sciclone/geograd/heather_data/imprecision/sch...,136826,"[120.99281798910403, 14.68076656583301]",120.992818,14.680767,-3.959799
50460,120.034409,123.13,/sciclone/geograd/heather_data/imprecision/sch...,136826,"[120.98710803469233, 14.672082746097777]",120.987108,14.672083,-81.206993
50461,119.442497,123.13,/sciclone/geograd/heather_data/imprecision/sch...,136826,"[120.98844991950588, 14.671782290066217]",120.988450,14.671782,-81.549507


In [14]:
use_coords, use_stats, use_fc = True, True, True

In [27]:
def prepare_features_targets(df):
    df["error"] = df["label"] - df["pred"]
    df['group_std_dev'] = df.groupby('DHSID')['pred'].transform('std')
    df['individual_deviation'] = df['pred'] - df.groupby('DHSID')['pred'].transform('mean')
    # df['individual_deviation'] = abs(df['individual_deviation'])
    df['group_mean'] = df.groupby('DHSID')['pred'].transform('mean')

    # # Initialize an empty list to store selected features
    # features = []
    
    # # Add statistical features if use_stats is True
    # if use_stats:
    #     features.append(df['group_std_dev'].values.reshape(-1, 1))
    #     features.append(df['individual_deviation'].values.reshape(-1, 1))
    #     features.append(df['group_mean'].values.reshape(-1, 1))
    
    # # Add coordinate features if use_coords is True
    # if use_coords:
    #     features.append(df['lon'].values.reshape(-1, 1))
    #     features.append(df['lat'].values.reshape(-1, 1))
    
    # # Add feature embeddings if use_fc is True
    # if use_fc:
    #     features.append(df['fc'].values.reshape(-1, 1))
    
    # # Stack the selected features horizontally to form X
    # if features:
    #     X = np.hstack(features)
    # else:
    #     raise ValueError("No features selected for the feature matrix.")
        
    # y = df['error'].values
    return df
    # return X, y

# Prepare training and testing data
# X_train, y_train = prepare_features_targets(train_df)
# X_test, y_test = prepare_features_targets(test_df)

In [38]:
t = prepare_features_targets(test_df)
t

,pred,label,name,DHSID,coords,lon,lat,fc,error,group_std_dev,individual_deviation,group_mean
0,162.402084,105.30,/sciclone/geograd/heather_data/imprecision/sch...,107022,"[120.62153863340414, 15.144364498536078]",120.621539,15.144364,4.462148,-57.102084,5.885643,2.264764,160.137320
1,152.662704,105.30,/sciclone/geograd/heather_data/imprecision/sch...,107022,"[120.62149201631127, 15.148910154310002]",120.621492,15.148910,4.560034,-47.362704,5.885643,-7.474615,160.137320
2,154.639465,105.30,/sciclone/geograd/heather_data/imprecision/sch...,107022,"[120.61545073309476, 15.140583949677787]",120.615451,15.140584,32.661892,-49.339465,5.885643,-5.497855,160.137320
3,158.953308,105.30,/sciclone/geograd/heather_data/imprecision/sch...,107022,"[120.61289533296045, 15.140260339858935]",120.612895,15.140260,33.107948,-53.653308,5.885643,-1.184012,160.137320
4,155.487823,105.30,/sciclone/geograd/heather_data/imprecision/sch...,107022,"[120.61574274638822, 15.137084619987599]",120.615743,15.137085,32.270683,-50.187823,5.885643,-4.649496,160.137320
...,...,...,...,...,...,...,...,...,...,...,...,...
50468,119.950241,103.49,/sciclone/geograd/heather_data/imprecision/sch...,136827,"[120.98575373790308, 14.687118257049514]",120.985754,14.687118,-3.959799,-16.460241,0.118156,0.070159,119.880082
50469,119.701080,103.49,/sciclone/geograd/heather_data/imprecision/sch...,136827,"[120.98343386365767, 14.68029533410494]",120.983434,14.680295,-3.959799,-16.211080,0.118156,-0.179002,119.880082
50470,119.846024,103.49,/sciclone/geograd/heather_data/imprecision/sch...,136827,"[120.98491348289478, 14.683843237416868]",120.984913,14.683843,-3.959799,-16.356024,0.118156,-0.034058,119.880082
50471,119.787369,103.49,/sciclone/geograd/heather_data/imprecision/sch...,136827,"[120.98536970045366, 14.68893022719709]",120.985370,14.688930,-3.959799,-16.297369,0.118156,-0.092713,119.880082


In [39]:
t.shape

(16831, 12)

In [40]:
t.dropna().shape

(16830, 12)

In [27]:
X_train.shape

(33642, 6)

In [16]:
import numpy as np

# Set a random seed for reproducibility
np.random.seed(42)

# Get 50 random indices from the range of X_test
indices = np.random.choice(X_train.shape[0], size=1000, replace=False)

# Sample the corresponding rows of X_test and y_test
X_test_sample = X_train[indices]
y_test_sample = y_train[indices]

In [17]:
# Train model
model = RandomForestRegressor(max_depth=10, min_samples_split=2, max_features='sqrt', min_samples_leaf=4)
model.fit(X_test_sample, y_test_sample)

RandomForestRegressor(max_depth=10, max_features='sqrt', min_samples_leaf=4)

In [37]:
    # Validate model
    y_train_pred = model.predict(X_train)
    # y_test_pred = model.predict(X_test)
    
    # # Calculate R2 scores for train and test
    # train_r2 = r2_score(y_train, y_train_pred)
    # test_r2 = r2_score(y_test, y_test_pred)

In [41]:
val_ids = pd.read_csv(nodups_preds)
val_ids["id"] = val_ids["name"].str.split("/").str[-1].str.split(".").str[0]
val_ids = list(val_ids["id"].unique())
val_ids[0:5]

['107022', '107027', '107028', '107029', '107032']

In [43]:
test_df["id"] = test_df["name"].str.split("/").str[-1].str.split(".").str[0]
test_df

,pred,label,name,DHSID,coords,lon,lat,fc,error,group_std_dev,individual_deviation,group_mean,id
0,162.402084,105.30,/sciclone/geograd/heather_data/imprecision/sch...,107022,"[120.62153863340414, 15.144364498536078]",120.621539,15.144364,4.462148,-57.102084,5.885643,2.264764,160.137320,107022_1
1,152.662704,105.30,/sciclone/geograd/heather_data/imprecision/sch...,107022,"[120.62149201631127, 15.148910154310002]",120.621492,15.148910,4.560034,-47.362704,5.885643,-7.474615,160.137320,107022_2
2,154.639465,105.30,/sciclone/geograd/heather_data/imprecision/sch...,107022,"[120.61545073309476, 15.140583949677787]",120.615451,15.140584,32.661892,-49.339465,5.885643,-5.497855,160.137320,107022_3
3,158.953308,105.30,/sciclone/geograd/heather_data/imprecision/sch...,107022,"[120.61289533296045, 15.140260339858935]",120.612895,15.140260,33.107948,-53.653308,5.885643,-1.184012,160.137320,107022_4
4,155.487823,105.30,/sciclone/geograd/heather_data/imprecision/sch...,107022,"[120.61574274638822, 15.137084619987599]",120.615743,15.137085,32.270683,-50.187823,5.885643,-4.649496,160.137320,107022_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50468,119.950241,103.49,/sciclone/geograd/heather_data/imprecision/sch...,136827,"[120.98575373790308, 14.687118257049514]",120.985754,14.687118,-3.959799,-16.460241,0.118156,0.070159,119.880082,136827_6
50469,119.701080,103.49,/sciclone/geograd/heather_data/imprecision/sch...,136827,"[120.98343386365767, 14.68029533410494]",120.983434,14.680295,-3.959799,-16.211080,0.118156,-0.179002,119.880082,136827_7
50470,119.846024,103.49,/sciclone/geograd/heather_data/imprecision/sch...,136827,"[120.98491348289478, 14.683843237416868]",120.984913,14.683843,-3.959799,-16.356024,0.118156,-0.034058,119.880082,136827_8
50471,119.787369,103.49,/sciclone/geograd/heather_data/imprecision/sch...,136827,"[120.98536970045366, 14.68893022719709]",120.985370,14.688930,-3.959799,-16.297369,0.118156,-0.092713,119.880082,136827_9
